In [1]:
# Standard library imports
import os
import time
from collections import OrderedDict

from grag.create_graph import (
    create_driver, 
    initialize_embeddings,
    initialize_llm,
    initialize_vector_store,
    create_vector_index,
    create_documents_from_df,
    create_batches,
    delete_all_nodes_and_indexes,
    embed_and_add_batch,
    qa_pipeline
)

# Third-party imports
import logging
from dotenv import load_dotenv
import polars as pl
from tqdm import tqdm

# LangChain imports
from langchain.chains import RetrievalQA


In [2]:
# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# Variables
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_URI = os.getenv("NEO4J_URI")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Define max tokens per batch
max_tokens_per_batch = 100000

In [4]:
CREATE_VECTOR_INDEX_CYPHER = """
CREATE VECTOR INDEX vector_index FOR (c:Chunk) ON (c.embedding)
OPTIONS {
  indexConfig:{
    `vector.dimensions`: 1536,
    `vector.hnsw.m`: 48,
    `vector.hnsw.ef_construction`: 200,
    `vector.quantization.enabled`: false,
    `vector.similarity_function`: 'cosine'
  }
};
"""

In [ ]:
# Import data
chunk_df = pl.read_parquet("/Users/borja/Documents/Somniumrema/projects/genai/grag/pipeline_outcomes/chunks_df.parquet")

# Import questions
questions = pl.read_csv("/Users/borja/Documents/Somniumrema/projects/genai/grag/data/acquired-qa-evaluation.csv",
                        encoding="latin")

In [4]:
# Initialize Neo4j driver
driver = create_driver(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)

# Delete all nodes and indexes
# delete_all_nodes_and_indexes(driver)

# Initialize embeddings
embeddings = initialize_embeddings()
        
# Initialize language model
llm = initialize_llm(model_name = "gpt-3.5-turbo", temperature = 0)
        
# Initialize vector store
vector_store = initialize_vector_store(
    embeddings=embeddings,
    uri=NEO4J_URI,
    user=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
        )

# Initialize the RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Simple concatenation of retrieved docs into the prompt
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 5 relevant documents
    return_source_documents=True
)

INFO:grag.create_graph:Neo4j driver created successfully.
INFO:grag.create_graph:Embeddings initialized with model: text-embedding-ada-002
INFO:grag.create_graph:Language model initialized with model: gpt-3.5-turbo
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:grag.create_graph:Neo4jVector store initialized with index: vector_index


In [65]:
# Initialize the RetrievalQA Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # Simple concatenation of retrieved docs into the prompt
    retriever=vector_store.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 5 relevant documents
    return_source_documents=True
)

In [8]:
# Create the vector index
create_vector_index(driver, CREATE_VECTOR_INDEX_CYPHER)

INFO:grag.create_graph:Index 'vector_index' created successfully.


In [9]:
# Create documents from DataFrame
documents = create_documents_from_df(chunk_df)
logger.info(f"Total documents created: {len(documents)}")

INFO:grag.create_graph:Starting to create Document objects from DataFrame.
INFO:grag.create_graph:Created 1304 Document objects.
INFO:__main__:Total documents created: 1304


In [10]:
# Split documents into manageable batches
batches = create_batches(documents, max_tokens_per_batch=max_tokens_per_batch)
total_batches = len(batches)
print(f"Total Batches to Process: {total_batches}")


# Process each batch
for i, batch in enumerate(tqdm(batches, desc="Processing Batches")):
    embed_and_add_batch(batch, embeddings, vector_store, batch_num=i+1)
    time.sleep(1)  # Sleep for 1 second between batches

Total Batches to Process: 45


Processing Batches:   0%|          | 0/45 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batch 1: Successfully added to Neo4jVector.


Processing Batches:   2%|▏         | 1/45 [00:05<04:02,  5.52s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batch 2: Successfully added to Neo4jVector.


...

Processing Batches:  96%|█████████▌| 43/45 [04:15<00:13,  6.83s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 3.039000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batch 44: Successfully added to Neo4jVector.


Processing Batches:  98%|█████████▊| 44/45 [04:23<00:07,  7.22s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO:openai._base_client:Retrying request to /embeddings in 1.683000 seconds
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Batch 45: Successfully added to Neo4jVector.


Processing Batches: 100%|██████████| 45/45 [04:30<00:00,  6.00s/it]


In [6]:
# Example usage
user_question = "When did Airbnb go public?"
answer, sources = qa_pipeline(qa, user_question)

print(f"Question: {user_question}")
print(f"Answer: {answer}\n")

print("Sources:")
unique_sources = OrderedDict()

for doc in sources:
    # Combine multiple metadata fields for a richer source description
    post_title = doc.metadata.get('post_title', 'No Title')
    series_number = doc.metadata.get('series_number', 'No Series Info')
    file_name = doc.metadata.get('file_name', 'No File Info')
    date = doc.metadata.get('blog_date', 'No Date Info')
    
    # Format the source string with context (file_name)
    source = f"Post title: {post_title} | Series number: {series_number} | File name: {file_name} | Blog date: {date}"
    
    # Use OrderedDict to maintain insertion order and uniqueness
    unique_sources[source] = None

# Print unique sources
for source in unique_sources.keys():
    print(f"- {source}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Question: When did Airbnb go public?
Answer: Airbnb went public on December 10, 2020.

Sources:
- Post title: Airbnb | Series number: Season 7, Episode 8 | File name: airbnb | Blog date: 2020-12-10
